# Improvements
- BlueConfig paths may not be changed here?
- Pull BlueConfig and reports with Unicore API

# Show Image 22222

In [ ]:
# %pylab inline
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# img=mpimg.imread('/mnt/5aa1d06c-d409-4d4a-8d4f-d90f51350047.jpg')
# imgplot = plt.imshow(img)
# plt.show()

# Check content of NFS

In [ ]:
!ls /mnt

In [ ]:
!cat /mnt/circuits/O1/20181114/user.target | grep Cell

# Pull BlueConfig

In [ ]:
LOCAL_DIR = '/home/jovyan/tmp/test_pull_unicore'
# !rm -drf $LOCAL_DIR
!mkdir -p $LOCAL_DIR
CIRCUIT_DIR = '/mnt/circuits/O1/20181114'

In [ ]:
# !pip install pyunicore --upgrade
import pyunicore.client as unicore_client
import pprint
from os.path import join
base = "https://hbp-unic.fz-juelich.de:7112/NUVLA/rest/core"
tr = unicore_client.Transport(oauth.get_token())

In [ ]:
# See all jobs
# client = unicore_client.Client(tr,base)
# all_jobs = client.get_jobs()
# job_selected = all_jobs[0]
# for job in all_jobs:
#     print(job.properties['name'])

In [ ]:
# Get one specific simulation
job_url = 'SIMULATION_URL'
job = unicore_client.Job(tr, job_url)
print('Simulation Name: {}'.format(job.properties['name']))
storage = job.working_dir
print('Files:')
files_list = storage.listdir()
pprint.pprint(files_list)

In [ ]:
# function to download the file and add it to kernel file system
def download_file_to_storage(file_name):
    x = files_list[file_name]
    file_content = x.raw().read()

    new_path = join(LOCAL_DIR, file_name)
    
    if file_name == 'BlueConfig':
        writable_content = file_content.replace('/mooc', '/mnt')
        writable_content = writable_content.replace('/io', LOCAL_DIR)
    else:
        writable_content = file_content

    with open(new_path, 'w') as fd:
        fd.write(writable_content)

    print('{} created!'.format(new_path))

In [ ]:
# pull the BlueConfig with Unicore
download_file_to_storage('BlueConfig')

In [ ]:
# find report
report = next((x for x in files_list if '_report_' in x), None)
download_file_to_storage(report)

In [ ]:
# pull the out.dat
download_file_to_storage('out.dat')

In [ ]:
!ls $LOCAL_DIR

# Explore Circuit

In [ ]:
# !pip install -q -i https://bbpteam.epfl.ch/repository/devpi/simple/ voxcell --force-reinstall
# !pip install -q -i https://bbpteam.epfl.ch/repository/devpi/simple/ bluepy --force-reinstall

In [ ]:
from voxcell import VoxelData, RegionMap
import bluepy
from bluepy.v2.enums import Cell
from voxcell.nexus.voxelbrain import Atlas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

In [ ]:
circuit = bluepy.Circuit(join(LOCAL_DIR, 'BlueConfig'))

In [ ]:
mtypes = circuit.v2.cells.mtypes
targets = ['mc'+str(idx) for idx in range(7)]

In [ ]:
composition = pd.DataFrame(index=mtypes, columns=targets)
for target in targets:
    for mtype in mtypes:
        composition[target][mtype] = len(circuit.v2.cells.ids(group={Cell.MTYPE: mtype, Cell.REGION: '@'+target+'.*'}))

composition.head()

In [ ]:
# Get the GID of miniColumns
minicolumns = ['mc{}_Column'.format(str(idx)) for idx in range(7)]
gids_by_mc = {}
for minicolum in minicolumns:
    mcX_values = circuit.v2.cells.get(minicolum).index.get_values()
    print('Name: {} - Cells: {} - GIDs: {}'.format(minicolum, len(mcX_values), mcX_values))
    gids_by_mc[minicolum] = mcX_values


# Connectivity

In [ ]:
def pair_coord_distance(circuit, pregid, postgid):
    p0 = circuit.v2.cells.get(pregid, [Cell.X, Cell.Y, Cell.Z]).values
    p1 = circuit.v2.cells.get(postgid, [Cell.X, Cell.Y, Cell.Z]).values
    return abs(p0-p1)

def connection_probability(circuit, pregids, postgids, zdist, xmin, xmax, nsample):
    counter = 0
    connected = 0
    exp_attempted = 0
    for idx in range(nsample):
        attempts = 0
        pregid = np.random.choice(pregids)
        # check if the cells are within the defined distance
        distant = True
        while distant:
            postgid = np.random.choice(postgids)
            # avoid to pick up the same gid
            if postgid == pregid:
                pass
            dx, _dy, dz = pair_coord_distance(circuit, pregid, postgid)
            if (dx < xmax) and (dx < xmax) and (dz < zdist):
                distant = False
            else:
                attempts += 1
            # avoid to enter in an infinit loop
            if attempts > 100:
                distant = False
                exp_attempted += 1
        if circuit.v2.stats.sample_pathway_synapse_count(1, pre=pregid, post=postgid) > 0:
                connected += 1
        if exp_attempted > 10:
            pass
    return connected/nsample

In [ ]:
bio_path = '/mnt/entities/bionames/20181114/connection_probability.txt'

In [ ]:
df = pd.read_csv(bio_path, sep='\t', index_col = False)

In [ ]:
df.head()

In [ ]:
# remove datapoint with n<10
df.drop([3, 11, 13, 17], inplace=True)

In [ ]:
nsample = 1000
nrepetitions = 3
xmax = 400
zdist = 100

In [ ]:
df['mod_mean'] = 0.0
df['mod_std'] = 0.0

In [ ]:
for index, row in df.iterrows():
    
    model_probability = np.zeros(nrepetitions)
    
    target = 'slice' + str(int(row['slice']))
    
    pre1 = circuit.v2.cells.ids(row['Pre'])
    pre2 = circuit.v2.cells.ids(target)
    pregids = np.intersect1d(pre1, pre2)
    
    post1 = circuit.v2.cells.ids(row['Post'])
    post2 = circuit.v2.cells.ids(target)
    postgids = np.intersect1d(post1, post2)
    
    if len(pregids)*len(pregids) == 0:
        print('missing mtype')
        pass
        
    for idx in range(nrepetitions):
        model_probability[idx] = connection_probability(circuit, pregids, postgids, zdist, row['xmin'], xmax, nsample)

    df.loc[index, 'mod_mean'] = model_probability.mean()
    df.loc[index, 'mod_std'] = model_probability.std()

    print(time.asctime(), row['Pre'], row['Post'], 'done')
    

# Analyze Reports

In [ ]:
from bluepy.v2 import Simulation, Circuit
from bluepy.v2.enums import Cell
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
# from PyPDF2 import PdfFileMerger, PdfFileReader
import os
import pandas as pd

In [ ]:
sim = Simulation(join(LOCAL_DIR, 'BlueConfig'))

In [ ]:
gids = sim.circuit.cells.ids(group={Cell.MTYPE: 'SP_PC', Cell.REGION: '@mc2.*'}, sample=100)

In [ ]:
fig, ax = plt.subplots()
# ax = sim.v2.plot.raster(gids, groupby='mtype', t_start=0)
ax = sim.plot.raster(gids, t_start=0)
# ax.legend()
timestamp = datetime.now().strftime('%Y%m%dT%H%M%S')
# fig.savefig('/home/romani/Desktop/rasterplot_' + timestamp + '.png')
plt.show()